# Part I: Monte Carlo ES

- Here you are given an executable that represents the Markov Decision Process. The executable is named [```MDP```](./MDP).

- You can query the number of states and actions of the MDP with ```./MDP states``` and ```./MDP actions```. The discount factor of the MDP can be obtained with ```./MDP gamma```.

- To start interacting with the MDP, run ```./MDP <starting state>```. At every iteration, the executable will display the current state and current return of the MDP, and ask you to choose an action, after which it will give a reward, and transition to a new state.

- You must implement the Monte Carlo ES algorithm that learns the optimal policy of the MDP by simulating episodes with exploring starts.


In [ ]:
'''
Solving an MDP with Monte Carlo ES (Exploring Starts) algorithm
This program uses subprocess library to operate the given enviroment (an .exe file) and obtain the optimal policy of this particular MDP
'''

In [190]:
import subprocess
import random
import numpy as np

In [191]:
class MonteCarloES:
    def __init__(self) -> None:
        process = subprocess.Popen(['./a.out', "states"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        stdout = stdout.decode('utf-8')
        self.num_states = int(stdout)
        process = subprocess.Popen(['./a.out', "actions"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        stdout = stdout.decode('utf-8')
        self.num_actions = int(stdout)
        process = subprocess.Popen(['./a.out', "gamma"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        stdout = stdout.decode('utf-8')
        self.gamma = float(stdout)
        self.Q = np.zeros((self.num_states, self.num_actions))
        self.policy = np.array(self.num_states)
        self.returns = [[[] for j in range(self.num_actions)] for i in range(self.num_states)]
        self.actions = []
        print("Number of states : ", self.num_states)
        print("Number of action : ", self.num_actions)
        print("Gamma : ", self.gamma)
    def action(self, episode_length=1000):
        self.actions = []
        state = random.randint(0,self.num_states-1)
        process = subprocess.Popen(['./a.out', str(state)], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for _ in range(episode_length):
            action = random.randint(0,4)
            self.actions.append(action)
            process.stdin.write((str(action)+'\n').encode('utf-8'))
        stdout, stderr = process.communicate()
        stdout = stdout.decode('utf-8')
        state_list = stdout.split("\n")[:-3:3][:-1]
        state_list = [int(states.split(":")[-1][1:]) for states in state_list]
        reward_list = stdout.split("\n")[2:-3:3]
        reward_list = [float(rewards.split(":")[-1][1:]) for rewards in reward_list]
        for j in range(1,len(reward_list)):
            reward_list[-j-1] += self.gamma*reward_list[-j]
        visited = [[0 for j in range(self.num_actions)] for i in range(self.num_states)]
        for j in range(len(self.actions)):
            if not any(0 in sublist for sublist in visited):
                break
            if not visited[state_list[j]][self.actions[j]]:
                visited[state_list[j]][self.actions[j]] = 1
                self.returns[state_list[j]][self.actions[j]].append(reward_list[j])
                self.Q[state_list[j]][self.actions[j]] = sum(self.returns[state_list[j]][self.actions[j]])/len(self.returns[state_list[j]][self.actions[j]])
        for j, state in enumerate(state_list):
            pass
    def train(self, iterations=1000, episode_length=100):
        for _ in range(iterations):
            self.action(episode_length)
        self.policy  = np.argmax(np.array(self.Q), axis=1)
    def showQ(self):
        print("\nq( s | a ) values for all states & action pairs")
        for i in range(self.num_states):
            print(*self.Q[i], sep=" ")
        print()
    def show_policy(self):
        print("States : Optimal actions")
        for i in range(self.num_states):
            print(i, " : ", self.policy[i])
    def evaluate(self, episode_length=100):
        state = random.randint(0,self.num_states-1)
        process = subprocess.Popen(['./a.out', str(state)], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for i in range(episode_length):
            state = random.randint(0,self.num_actions-1)
            process.stdin.write((str(state)+"\n").encode('utf-8'))
        stdout, stderr = process.communicate()
        stdout = stdout.decode('utf-8')
        value = stdout.split("\n")[-3].split(":")[-1][1:]
        print(f"\nFinal return by policy after an episode of length {episode_length}")
        print(value)

In [192]:
obj = MonteCarloES()
obj.train()
obj.showQ()
obj.show_policy()
obj.evaluate()

Number of states :  10
Number of action :  5
Gamma :  0.75

q( s | a ) values for all states & action pairs
-0.42273012472566246 -0.7045569931349105 -0.07574218242754943 -0.20957586518466448 -0.20473862357551462
0.06018118893736839 -0.7022091712794746 0.2892273414944502 -0.6566501341662108 -0.1569936495018607
0.9029657617242225 -0.4760514179437689 0.2139081024423429 -0.2900415846071898 -0.9331258582565156
-0.38479299280745644 -0.3117918134732012 -0.7011903776190129 0.09414301096148749 0.14916020938681246
0.11435157332304996 -0.5618629418520863 -0.3554294365062332 0.42339781900182716 -0.8788404961289737
-0.3950244984071992 -0.13083771763624447 -0.50112543458138 -0.28550886122375907 -0.6078089630433554
0.4331553759255612 -0.03138320480640642 0.17543666121777204 -0.4699798377929238 -0.6161913682290096
-0.20896874571112237 -0.3689946976400283 0.01645003893273207 -1.2351503568241327 -0.2184958179508261
-0.0546690550576868 -0.10468650189759063 0.5384559443989935 -0.6124941650788289 0.4022217